# Schedule Recommender System

Generates semester course recommendations for students based on program requirements, prerequisites, and preferences.

**Quick Start:** Run all cells from top to bottom (Cell → Run All)

In [ ]:
import pandas as pd
import sys
import os
from sqlalchemy.orm import Session

# Add root path for shared modules (notebook runs in Docker at /notebook, shared at /shared)
if os.path.exists('/shared'):
    sys.path.insert(0, '/')

from Database.database import (
    get_db_session, engine, check_db_connection, verify_tables_exist, reflect_referenced_tables
)
from Database.models import RecommendationResult
from Database.db_helpers import create_record

# Reflect referenced tables (like time_slots) so SQLAlchemy can validate foreign keys
reflect_referenced_tables()

# Import shared recommender components
from shared.semester_scheduler import SemesterScheduler
from shared.recommender_helpers import load_data_from_db


## 1. Load Data


In [2]:
# Verify database connection
print("Checking database connection...")
if not check_db_connection():
    raise ConnectionError("Cannot connect to database. Please check your DATABASE_URL.")

# Verify required tables exist
print("\nVerifying database tables...")
table_status = verify_tables_exist()
for table, exists in table_status.items():
    if not table.endswith('_count'):
        count = table_status.get(f"{table}_count", 0)
        status = "✓" if exists else "✗"
        print(f"{status} {table}: {'exists' if exists else 'missing'} ({count} rows)" if exists else f"{status} {table}: missing")

if not all(table_status.get(t, False) for t in ['students', 'courses', 'sections']):
    raise ValueError("Required tables are missing. Please ensure the database is initialized.")

# Get database session and load data using shared function
db = get_db_session()
data = load_data_from_db(engine, current_year=2025, current_semester='Fall')

print("\n✓ Data loaded successfully!")
print(f"  Students: {len(data['students'])}")
print(f"  Courses: {len(data['courses'])}")
print(f"  Sections: {len(data['sections'])}")
print(f"  Takes records: {len(data['takes'])}")
print(f"  Programs: {len(data['programs'])}")


Checking database connection...
✓ Database connection successful!

Verifying database tables...
✓ students: exists (10 rows)
✓ courses: exists (51 rows)
✓ sections: exists (51 rows)
✓ takes: exists (140 rows)
✓ prerequisites: exists (7 rows)
✓ programs: exists (3 rows)
✓ hascourse: exists (60 rows)
✓ time_slots: exists (468 rows)

✓ Data loaded successfully!
  Students: 10
  Courses: 51
  Sections: 51
  Takes records: 140
  Programs: 3


In [3]:
# Initialize scheduler using shared SemesterScheduler
scheduler = SemesterScheduler(data, current_year=2025, current_semester='Fall')
print("✓ Recommender initialized!")


✓ Recommender initialized!


## 2. Generate Recommendations


In [4]:
# Generate and save recommendations for all students

def save_recommendations_to_db(db: Session, scheduler, student_id: int, recommendations: list, 
                                 model_version: str = 'semester_scheduler_v1', 
                                 time_preference: str = 'any',
                                 semester: str = 'Fall',
                                 year: int = 2025):
    """Save semester recommendations to database."""
    student_id = int(student_id)
    
    if not recommendations:
        print(f"  ⚠️  No recommendations to save for student {student_id}")
        return
    
    saved_count = 0
    for slot_num, rec in enumerate(recommendations, 1):
        section_id = int(rec['section_id'])
        time_slot_id = scheduler.section_to_timeslot.get(section_id)
        why_recommended_str = ', '.join(rec.get('why_recommended', []))
        
        result_data = {
            'student_id': student_id,
            'course_id': int(rec['course_id']),
            'recommended_section_id': section_id,
            'course_name': rec['course_name'],
            'cluster': rec.get('cluster', ''),
            'credits': int(rec.get('credits', 0)),
            'time_slot': int(time_slot_id) if time_slot_id is not None else None,
            'recommendation_score': str(rec.get('score', '1.0')),
            'why_recommended': why_recommended_str,
            'slot_number': slot_num,
            'model_version': model_version,
            'time_preference': time_preference,
            'semester': semester,
            'year': year
        }
        try:
            create_record(db, RecommendationResult, result_data)
            saved_count += 1
        except Exception as e:
            print(f"  ⚠️  Failed to save recommendation {slot_num}: {e}")
            raise
    
    if saved_count > 0:
        print(f"  ✓ Saved {saved_count} recommendations to database for student {student_id}")


In [5]:
# Generate recommendations for all students
if len(data['students']) > 0:
    all_students = data['students']
    print(f"Generating recommendations for {len(all_students)} students...\n")
    
    # Process all students
    for idx, student_row in all_students.iterrows():
        student_id = int(student_row['student_id'])
        student_name = student_row['student_name']
        program_name = student_row['program_name']
        
        print(f"{'='*80}")
        print(f"Student: {student_name} (ID: {student_id})")
        print(f"Program: {program_name}")
        
        # Get student standing and credits
        credits = scheduler.get_student_credits(student_id)
        standing = scheduler.get_student_standing(student_id)
        print(f"Credits Completed: {credits}")
        print(f"Standing: {standing}")
        
        # Get cluster profile
        cluster_profile = scheduler.get_student_cluster_profile(student_id)
        print(f"Cluster Profile: {cluster_profile}")
        
        # Get remaining Gen-Ed requirements
        gened_remaining = scheduler.get_remaining_gened_requirements(student_id)
        print(f"Remaining Gen-Ed Requirements:")
        for group, needed in gened_remaining.items():
            print(f"  Group {group}: {needed} courses needed")
        
        # Generate semester recommendations
        print(f"\nSEMESTER RECOMMENDATIONS (Time Preference: any)")
        recommendations = scheduler.recommend_semester(student_id, time_preference='any')
        
        if recommendations:
            print(f"Recommended {len(recommendations)} courses:")
            for i, rec in enumerate(recommendations, 1):
                print(f"  {i}. {rec['course_name']} (ID: {rec['course_id']}, Credits: {rec['credits']}, Cluster: {rec['cluster']})")
            
            # Save recommendations to database
            try:
                save_recommendations_to_db(db, scheduler, student_id, recommendations, 
                                          'semester_scheduler_v1', 'any', 'Fall', 2025)
            except Exception as e:
                print(f"  ⚠️  Warning: Could not save recommendations to database: {e}")
                print(f"     Make sure the database is initialized (run 'docker compose up')")
        else:
            print("No recommendations available.")
            print("  (Student may have completed all required courses or no sections available)")
        
        print()  # Empty line between students
    
    print(f"{'='*80}")
    print(f"Completed processing {len(all_students)} students.")
else:
    print("No students found in database.")


Generating recommendations for 10 students...

Student: Armen (ID: 1)
Program: BSDS
Credits Completed: 61
Standing: Junior
Cluster Profile: {9: 2, 10: 3, 12: 3, 8: 1, 7: 1, 13: 1, 14: 2, 15: 3, 16: 2}
Remaining Gen-Ed Requirements:
  Group A: 0 courses needed
  Group B: 0 courses needed
  Group C: 0 courses needed

SEMESTER RECOMMENDATIONS (Time Preference: any)
Recommended 3 courses:
  1. DS 116 Data Visualization (ID: 18, Credits: 4, Cluster: Core)
  2. CS 100 Calculus 1 (ID: 1, Credits: 3, Cluster: Core)
  3. CS 111 Discrete Math (ID: 6, Credits: 3, Cluster: Core)
  ⚠️  Failed to save recommendation 1: Database error: Foreign key associated with column 'recommendation_results.time_slot' could not find table 'time_slots' with which to generate a foreign key to target column 'time_slot_id'
  ⚠️  Warning: Could not save recommendations to database: Database error: Foreign key associated with column 'recommendation_results.time_slot' could not find table 'time_slots' with which to gener

## 3. View Results

In [6]:
# Display saved recommendations

print("="*80)
print("SAVED RECOMMENDATIONS IN DATABASE")
print("="*80)

try:
    # Query all recommendations from database
    recommendations_df = pd.read_sql_table('recommendation_results', engine)
    
    if len(recommendations_df) == 0:
        print("\n⚠️  No recommendations found in database.")
        print("   Make sure you've run Cell 7 to generate and save recommendations for all students.")
    else:
        print(f"\n✓ Found {len(recommendations_df)} saved recommendations")
        print(f"  Total students with recommendations: {recommendations_df['student_id'].nunique()}")
        
        # Group by student
        for student_id in sorted(recommendations_df['student_id'].unique()):
            student_recs = recommendations_df[recommendations_df['student_id'] == student_id]
            student_name = data['students'][data['students']['student_id'] == student_id]['student_name'].values
            student_name = student_name[0] if len(student_name) > 0 else f"Student {student_id}"
            
            print(f"\n  Student: {student_name} (ID: {student_id})")
            print(f"  Recommendations: {len(student_recs)}")
            for _, rec in student_recs.iterrows():
                print(f"    - {rec['course_name']} (Slot {rec['slot_number']}, Credits: {rec['credits']}, Cluster: {rec['cluster']})")
        
        # Summary statistics
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS")
        print(f"{'='*80}")
        print(f"Total recommendations: {len(recommendations_df)}")
        print(f"Students with recommendations: {recommendations_df['student_id'].nunique()}")
        print(f"Average recommendations per student: {len(recommendations_df) / recommendations_df['student_id'].nunique():.1f}")
        print(f"Model version: {recommendations_df['model_version'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        print(f"Semester: {recommendations_df['semester'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        print(f"Year: {recommendations_df['year'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        
except Exception as e:
    print(f"\n⚠️  Error querying database: {e}")
    print("   Make sure you've run the previous cell to generate recommendations.")


SAVED RECOMMENDATIONS IN DATABASE

✓ Found 33 saved recommendations
  Total students with recommendations: 9

  Student: Armen (ID: 1)
  Recommendations: 1
    - CS 111 Discrete Math (Slot 1, Credits: 3, Cluster: Core)

  Student: Alla (ID: 2)
  Recommendations: 3
    - CS 101 Calculus 2 (Slot 1, Credits: 3, Cluster: Core)
    - CS 111 Discrete Math (Slot 2, Credits: 3, Cluster: Core)
    - BUS 101 Introduction to Business (Slot 3, Credits: 3, Cluster: Core)

  Student: Levon (ID: 3)
  Recommendations: 4
    - DS 116 Data Visualization (Slot 1, Credits: 4, Cluster: Core)
    - CS 100 Calculus 1 (Slot 2, Credits: 3, Cluster: Core)
    - CS 111 Discrete Math (Slot 3, Credits: 3, Cluster: Core)
    - FND 221 Armenian History 1 (Slot 4, Credits: 3, Cluster: Foundation)

  Student: Marieta (ID: 4)
  Recommendations: 4
    - CS 100 Calculus 1 (Slot 1, Credits: 3, Cluster: Core)
    - CS 111 Discrete Math (Slot 2, Credits: 3, Cluster: Core)
    - CHSS 170 Religion in America (Slot 3, Credits: